In [1]:
import plotly
import plotly.graph_objs as go
import torch
from torch.utils.data import DataLoader
from experiment import Net, InitDataset, CreateModel, LoadCheckpoint
from yaml import safe_load
import torch.nn as nn

exphash = '9e8920f6e'

exp = 'results/%s' % exphash
with open(exp) as f:
  h = safe_load(f.read())
h = h['exp_info']  

h['dataset_split'][1] = .1
ds = InitDataset(h)


model = CreateModel(h['hyperparameters'])
LoadCheckpoint(model, 'models/%s'%exphash)

model = model.cuda()


[18:54:47] Loaded model: <All keys matched successfully>


In [2]:
val_loader = DataLoader(ds[1], batch_size=32, shuffle=False)

In [3]:
idx_to_char_map = {v:k for k,v in torch.load('../data/char_to_idx_map.pt').items()}

In [4]:
idx_to_char_map[0] = '_'

In [5]:
Torch2Py = lambda x: x.cpu().numpy().tolist()

In [7]:
def bin_acc_fn(outputs, labels):
  binary = outputs>.5
  right = binary==labels
  return right.float()

BCE_loss = nn.BCELoss(reduction='none')
def bce_loss_fn(outputs, labels):
  return BCE_loss(outputs, labels)

def Validate(val_loader, model):
  device = next(model.parameters()).device
  cum_acc = 0
  cum_bce_loss = 0

  inputs_ = []
  losses_ = []
  guesses_ = []
  rights_ = []
  labels_ = []
  for i, data in enumerate(val_loader):
    data = {k: d.to(device) for k,d in data.items()}
    inputs = data['char_encoded_seqs']
    inputs_ += Torch2Py(inputs)
    labels = data['token_start_positions']
    labels_ += Torch2Py(labels)

    with torch.no_grad():
      outputs = model(inputs)
      guesses_ += Torch2Py(outputs)
      bce_loss = bce_loss_fn(outputs, labels)
      losses_ += Torch2Py(bce_loss)
      cum_bce_loss += bce_loss.mean()

      acc = bin_acc_fn(outputs, labels)
      rights_ += Torch2Py(acc)
      cum_acc += acc.mean()

  print(cum_acc/(i+1), cum_bce_loss/(i+1))
  return inputs_, losses_, rights_, labels_, guesses_

In [8]:
inputs, losses, rights, labels, guesses = Validate(val_loader, model)

tensor(0.9914, device='cuda:0') tensor(0.0373, device='cuda:0')


In [9]:
def GetChars(encoded):
  s = ""
  for c in encoded:
    s+= idx_to_char_map[c]
  return s.replace('\x00','_')

In [10]:
GetChars(inputs[0])

'- not to mention the exaggerated moans during the love-making and the fact that the couple\'s relationship never convinces for a second (if it\'s supposed to evoke "l\'amour fou", it\'s a pathetic attempt)! the subplot involving marini\'s gay brother\'s desperation over unrequited love for an imprisoned stud (which is all magically settled by the end) is merely boring.___________________'

In [11]:
length = 200
for i, r in enumerate(rights):
  if sum(r)<len(r) and i%10==0:
    # underscore above the letter if a tok start was missed
    print("".join('_' if labels[i][j]==1 and x!=1 else ' ' for j, x in enumerate(rights[i][:length])))
    print("".join(str(int(guesses[i][j]*10)) if x!=1 else ' ' for j, x in enumerate(rights[i][:length])))    
    print(GetChars(inputs[i])[:length])
    print("".join(' ' if x==1 else 'X' for x in rights[i][:length]))
    print()

                                                                                                                                                                            _                           
                                                                                                                                                                            0                           
- not to mention the exaggerated moans during the love-making and the fact that the couple's relationship never convinces for a second (if it's supposed to evoke "l'amour fou", it's a pathetic attempt
                                                                                                                                                                            X                           

                            _   _          _                                                                                               _                                                       

                                                                                                                                                                                             _       _ _
                                                                                                                                                                                             0       191
with this film. no comedy, no bunch of stupid jokes mixed together. we can find a correct mixture of little stories that happen in a workers town (the one the directors are from, l'hospitalet de llobr
                                                                                                                                                                                             X       XXX

                                                                                  _                                                                                                                


                                                                                                0                                                                   0                                   
the humanity... aside from some highly derivative dialog (you'll notice some albert brooks-esquire lines and deliveries in there) there is a scene involving lime jell-o that will have you screaming, "
                                                                                                X                                                                   X                                   

                                                                                                                                                _                                                       
                                                                                                                                                0                                                 